Importing all needed files and Libaries


In [ ]:
!pip -q install pandas numpy chardet openpyxl pyjanitor rapidfuzz unidecode seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os,re, io
import janitor  # pyjanitor
from rapidfuzz import fuzz, process
from unidecode import unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.2/223.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 18.8 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Importing datasets that I will be working with.

In [ ]:
# --- Define file paths ---
postdoc_All     = "/content/RU Health Postdocs as of 8-7-24 all(Sheet2).csv"
postdoc_noPharmD = "/content/RU Health Postdocs as of 8-7-24 no PharmD(Sheet2).csv"
path_rbhs       = "/content/RBHS_Fac_Staff_2024_Nick's File(Sheet2).csv"
path_pwac       = "/content/PWAC Resident Info 2024(PWAC Emails).csv"
path_AllFac     = "/content/All Faculty Report for Wellness Survey_2024-06-28(ALL Faculy forWellness Survey).csv"



/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

Quick preview of the columns of the data sets

In [ ]:
# ---- Helper Function to Safely Read CSVs ----
def safe_read_csv(path):
    """Reads a CSV safely with latin1 encoding (handles special chars)."""
    try:
        df = pd.read_csv(path, encoding="latin1")
        print(f"✅ Loaded: {os.path.basename(path)} | Shape: {df.shape}")
        return df
    except Exception as e:
        print(f"❌ Error loading {os.path.basename(path)}: {e}")
        return pd.DataFrame()

# ---- Load All Datasets ----
df_postdoc_All      = safe_read_csv(postdoc_All)
df_postdoc_noPharmD = safe_read_csv(postdoc_noPharmD)
df_rbhs             = safe_read_csv(path_rbhs)
df_pwac             = safe_read_csv(path_pwac)
df_AllFac           = safe_read_csv(path_AllFac)

# ---- Display Column Names for Each Dataset ----
print("\n📋 Columns in Each Dataset:")
datasets = {
    "Postdoc (All)": df_postdoc_All,
    "Postdoc (No PharmD)": df_postdoc_noPharmD,
    "RBHS Faculty/Staff": df_rbhs,
    "PWAC Residents": df_pwac,
    "All Faculty Report": df_AllFac
}

for name, df in datasets.items():
    print(f"\n{name}:")
    print(list(df.columns))

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

✅ Loaded: RU Health Postdocs as of 8-7-24 all(Sheet2).csv | Shape: (615, 13)
✅ Loaded: RU Health Postdocs as of 8-7-24 no PharmD(Sheet2).csv | Shape: (246, 13)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

✅ Loaded: RBHS_Fac_Staff_2024_Nick's File(Sheet2).csv | Shape: (10825, 8)
✅ Loaded: PWAC Resident Info 2024(PWAC Emails).csv | Shape: (1785, 17)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


✅ Loaded: All Faculty Report for Wellness Survey_2024-06-28(ALL Faculy forWellness Survey).csv | Shape: (2599, 39)

📋 Columns in Each Dataset:

Postdoc (All):
['Name', 'Net ID', 'Hire Dt', ' Reporting Campus', 'School/Div', 'Department', 'Location', 'Empl Class', 'Empl Class Name', 'Title', 'FTE', 'Empl Group', 'Email']

Postdoc (No PharmD):
['Name', 'Net ID', 'Hire Dt', ' Reporting Campus', 'School/Div', 'Department', 'Location', 'Empl Class', 'Empl Class Name', 'Title', 'FTE', 'Empl Group', 'Email']

RBHS Faculty/Staff:
['FirstName', 'LastName', 'Email', 'ETyp', 'Title', 'NetID', 'DeptNm', 'role']

PWAC Residents:
['Institution', 'Current Program', 'First Name', 'Last Name', 'Middle Name', 'Status', 'Work Roles', 'Program Start Date', 'Program End Date', 'Netid', 'Post Graduate Year', 'RMS ID Number', 'Organizational Email', 'Primary Email Address', 'Email Addresses Work Email', 'Email Addresses Home Address', 'Email Addresses Work Address']

All Faculty Report:
['Rutgers Biomedical 

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

Cleaning postDoc(All) & postDoc(noPharmD)

In [1]:
# ============================
# Clean + Dedupe POSTDOC files
# ============================

# (1) Imports
import pandas as pd
import numpy as np

# (2) Paths (your filenames)
postdoc_All      = "/content/RU Health Postdocs as of 8-7-24 all(Sheet2).csv"
postdoc_noPharmD = "/content/RU Health Postdocs as of 8-7-24 no PharmD(Sheet2).csv"

# (3) Load both CSVs with a safe encoding
pd_all = pd.read_csv(postdoc_All, encoding="latin1")
pd_no  = pd.read_csv(postdoc_noPharmD, encoding="latin1")

# (4) Combine and tidy headers
df = pd.concat([pd_all, pd_no], ignore_index=True)
df.columns = df.columns.str.strip()

# (5) Normalize key fields
if "Net ID" in df.columns:
    df["Net ID"] = df["Net ID"].astype(str).str.strip().str.lower().replace({"nan": pd.NA})
if "Email" in df.columns:
    df["Email"] = df["Email"].astype(str).str.strip().str.lower().replace({"nan": pd.NA})

# (6) Robust name split from a single "Name" column
def split_fullname_to_first_last(name_series: pd.Series):
    name = name_series.astype(str).str.strip().replace({"nan": pd.NA})
    has_comma = name.str.contains(",", na=False)

    first = pd.Series(index=name.index, dtype=object)
    last  = pd.Series(index=name.index, dtype=object)

    # Case A: "Last, First Middle..."
    part = name[has_comma].str.split(",", n=1, expand=True)
    if not part.empty:
        last.loc[has_comma]  = part[0].str.strip()
        rest = part[1].str.strip()
        first.loc[has_comma] = rest.str.split(r"\s+", n=1, expand=True)[0]

    # Case B: "First Middle Last"
    no_comma = ~has_comma
    part2 = name[no_comma].str.split(r"\s+", expand=True)
    if not part2.empty:
        first.loc[no_comma] = part2[0]
        last.loc[no_comma]  = part2.stack().groupby(level=0).last()

    return first, last

first, last = split_fullname_to_first_last(df["Name"]) if "Name" in df.columns else (pd.Series(dtype=object), pd.Series(dtype=object))

# (7) Build the cleaned output (prefixed HR columns to avoid future collisions)
out = pd.DataFrame({
    "netid":                      df.get("Net ID", pd.NA),
    "email":                      df.get("Email", pd.NA),
    "firstname":                  first,
    "lastname":                   last,
    "postdoc_title":              df.get("Title", pd.NA),
    "postdoc_department":         df.get("Department", pd.NA),
    "postdoc_reporting_campus":   df.get("Reporting Campus", pd.NA),
    "postdoc_school_div":         df.get("School/Div", pd.NA),
    "postdoc_location":           df.get("Location", pd.NA),
    "postdoc_empl_class":         df.get("Empl Class", pd.NA),
    "postdoc_empl_class_name":    df.get("Empl Class Name", pd.NA),
    "postdoc_empl_group":         df.get("Empl Group", pd.NA),
    "postdoc_fte":                pd.to_numeric(df.get("FTE", pd.NA), errors="coerce"),
    "postdoc_hire_date":          pd.to_datetime(df.get("Hire Dt", pd.NA), errors="coerce"),
    "source":                     "postdocs_all+noPharmD"
})

# (8) Drop rows with no identifying info at all
out = out[~(
    out["netid"].isna() &
    out["email"].isna() &
    out["firstname"].isna() &
    out["lastname"].isna()
)].copy()

# (9) Deduplicate:
#  - keep first occurrence for a given netid (when netid exists)
#  - for rows without netid, dedupe by email
has_netid = out["netid"].notna()
d_with_netid = out[has_netid].drop_duplicates(subset=["netid"], keep="first")
d_no_netid   = out[~has_netid]
if "email" in d_no_netid:
    d_no_netid = d_no_netid.drop_duplicates(subset=["email"], keep="first")

postdocs_clean = pd.concat([d_with_netid, d_no_netid], ignore_index=True)
postdocs_clean = postdocs_clean.drop_duplicates().reset_index(drop=True)

# (10) Order columns nicely
ID_COLS  = ["netid", "email", "firstname", "lastname"]
OTHER    = [c for c in postdocs_clean.columns if c not in ID_COLS]
postdocs_clean = postdocs_clean[ID_COLS + OTHER]

# (11) Save + summary
save_path = "/content/postdocs_cleaned.xlsx"
postdocs_clean.to_excel(save_path, index=False)

print("✅ Cleaned Postdocs shape:", postdocs_clean.shape)
print("💾 Saved to:", save_path)
postdocs_clean.head(10)


✅ Cleaned Postdocs shape: (591, 15)
💾 Saved to: /content/postdocs_cleaned.xlsx


,netid,email,firstname,lastname,postdoc_title,postdoc_department,postdoc_reporting_campus,postdoc_school_div,postdoc_location,postdoc_empl_class,postdoc_empl_class_name,postdoc_empl_group,postdoc_fte,postdoc_hire_date,source
0,aa1330,aa1330@rutgers.edu,Anuoluwapo,Adedeji,POST DOCTORAL ASSOC,Pharm-Pharmacy Practice&Admin,RBHS,Ernest Mario School - Pharmacy,New Brunswick,1,Regular FT/PT,Post Docs - Regular Salaried,1,2024-07-01,postdocs_all+noPharmD
1,aa2253,aa2253@rutgers.edu,Abigail,Armstrong,POST DOC FELLOW,CABM-RBHS,RBHS,CABM,New Brunswick,1,Regular FT/PT,Post Docs - Regular Salaried,1,2022-09-01,postdocs_all+noPharmD
2,aa3051,aa3051@rutgers.edu,Arevik,Abramyan,POST DOC FELLOW,RWJ-Neurosurgery,RBHS,RWJ-Finance Office,New Brunswick,1,Regular FT/PT,Post Docs - Regular Salaried,1,2023-10-02,postdocs_all+noPharmD
3,aac312,aac312@rutgers.edu,Angel,Choe,POST DOCTORAL ASSOC,Pharm-Pharmacy Practice&Admin,RBHS,Ernest Mario School - Pharmacy,New Brunswick,1,Regular FT/PT,Post Docs - Regular Salaried,1,2023-07-01,postdocs_all+noPharmD
4,aal204,aal204@rutgers.edu,Amie,Lette,POST DOCTORAL ASSOC,Pharm-Pharmacy Practice&Admin,RBHS,Ernest Mario School - Pharmacy,New Brunswick,1,Regular FT/PT,Post Docs - Regular Salaried,1,2023-07-01,postdocs_all+noPharmD
5,aao132,aao132@rutgers.edu,Abodunrin,Ojetola,POST DOC FELLOW,RWJ-Pharmacology-PISC,RBHS,RWJ-Finance Office,New Brunswick,1,Regular FT/PT,Post Docs - Regular Salaried,1,2023-12-12,postdocs_all+noPharmD
6,aar321,aar321@rutgers.edu,Andre,Rosario,POST DOC FELLOW,College of Nursing,RBHS,SN-Admin (Dean's Office),Newark,1,Regular FT/PT,Post Docs - Regular Salaried,1,2024-07-01,postdocs_all+noPharmD
7,aas529,aas529@rutgers.edu,Adejumoke,Shofoluwe,POST DOCTORAL ASSOC,Pharm-Pharmacy Practice&Admin,RBHS,Ernest Mario School - Pharmacy,New Brunswick,1,Regular FT/PT,Post Docs - Regular Salaried,1,2023-07-01,postdocs_all+noPharmD
8,aav106,aav106@rutgers.edu,Ashley,Volpe,POST DOCTORAL ASSOC,Pharm-Pharmacy Practice&Admin,RBHS,Ernest Mario School - Pharmacy,New Brunswick,1,Regular FT/PT,Post Docs - Regular Salaried,1,2023-07-01,postdocs_all+noPharmD
9,ab2331,ab2331@rutgers.edu,Ashis,Biswas,POST DOC FELLOW,NJMS-PHRI CENTER,RBHS,NJMS-Dean's Office,Newark,1,Regular FT/PT,Post Docs - Regular Salaried,1,2021-09-20,postdocs_all+noPharmD


Cleaning RBHS File

In [2]:
# ============================
# Clean + Dedupe RBHS Faculty/Staff file
# ============================

import pandas as pd
import numpy as np

# (1) Path to your RBHS dataset
path_rbhs = "/content/RBHS_Fac_Staff_2024_Nick's File(Sheet2).csv"

# (2) Load the CSV safely with encoding fallback
rbhs_raw = pd.read_csv(path_rbhs, encoding="latin1")
rbhs_raw.columns = rbhs_raw.columns.str.strip()  # remove trailing spaces
print(f"✅ Loaded RBHS file | Shape: {rbhs_raw.shape}")

# (3) Normalize key fields
if "NetID" in rbhs_raw.columns:
    rbhs_raw["NetID"] = rbhs_raw["NetID"].astype(str).str.strip().str.lower().replace({"nan": pd.NA})
if "Email" in rbhs_raw.columns:
    rbhs_raw["Email"] = rbhs_raw["Email"].astype(str).str.strip().str.lower().replace({"nan": pd.NA})

# (4) Handle potential name columns
# Some files use 'FirstName'/'LastName', others might have variations
first = rbhs_raw.get("FirstName", pd.Series(dtype=object)).astype(str).str.strip().replace({"nan": pd.NA})
last  = rbhs_raw.get("LastName",  pd.Series(dtype=object)).astype(str).str.strip().replace({"nan": pd.NA})

# (5) Build cleaned DataFrame with clear prefixes
rbhs_clean = pd.DataFrame({
    "netid":            rbhs_raw.get("NetID", pd.NA),
    "email":            rbhs_raw.get("Email", pd.NA),
    "firstname":        first,
    "lastname":         last,
    "rbhs_title":       rbhs_raw.get("Title", pd.NA),
    "rbhs_department":  rbhs_raw.get("DeptNm", pd.NA),
    "rbhs_employee_type": rbhs_raw.get("ETyp", pd.NA),
    "rbhs_role":        rbhs_raw.get("role", pd.NA),
    "source":           "rbhs_faculty_staff"
})

# (6) Drop rows that have no identifying info (no netid + no email + no name)
rbhs_clean = rbhs_clean[~(
    rbhs_clean["netid"].isna() &
    rbhs_clean["email"].isna() &
    rbhs_clean["firstname"].isna() &
    rbhs_clean["lastname"].isna()
)].copy()

# (7) Deduplicate:
#     - Prefer NetID as unique key
#     - Fallback to Email for missing NetIDs
has_netid = rbhs_clean["netid"].notna()
d_with_netid = rbhs_clean[has_netid].drop_duplicates(subset=["netid"], keep="first")
d_no_netid   = rbhs_clean[~has_netid]
if "email" in d_no_netid:
    d_no_netid = d_no_netid.drop_duplicates(subset=["email"], keep="first")

rbhs_clean = pd.concat([d_with_netid, d_no_netid], ignore_index=True)
rbhs_clean = rbhs_clean.drop_duplicates().reset_index(drop=True)

# (8) Reorder columns to match Postdocs structure
ID_COLS = ["netid", "email", "firstname", "lastname"]
META    = [c for c in rbhs_clean.columns if c not in ID_COLS]
rbhs_clean = rbhs_clean[ID_COLS + META]

# (9) Save and preview
save_path = "/content/rbhs_cleaned.xlsx"
rbhs_clean.to_excel(save_path, index=False)

print("✅ Cleaned RBHS shape:", rbhs_clean.shape)
print("💾 Saved to:", save_path)
rbhs_clean.head(10)


✅ Loaded RBHS file | Shape: (10825, 8)
✅ Cleaned RBHS shape: (10425, 9)
💾 Saved to: /content/rbhs_cleaned.xlsx


,netid,email,firstname,lastname,rbhs_title,rbhs_department,rbhs_employee_type,rbhs_role,source
0,ksa53,ksa53@rbhs.rutgers.edu,Kyle S.,Aaronson,ASSOC VICE CHANCELLOR FINANCE,RBHS - Finance & Admin,1,STAFF,rbhs_faculty_staff
1,aasala,iyabo.aasa@rutgers.edu,Iyabo,Aasa,LPN,UCHC-South Woods Psn-Ancillary,1,STAFF,rbhs_faculty_staff
2,abadgb,abadgb@rutgers.edu,Grace,Abad,ADVANCED PRACTICE NURSE PD,UBHC-Acute Psych Services,4,STAFF,rbhs_faculty_staff
3,abadva,valori.abad@rutgers.edu,Valori,Abad,REGIONAL NURSE ADMINISTRATOR,SN-FXB Center-Chp Program,1,STAFF,rbhs_faculty_staff
4,ea626,ea626@rutgers.edu,Elizabeth,Abadiotakis,SHP-PD,SHP-Psych Rehab & Counsel Pro,4,STAFF,rbhs_faculty_staff
5,abasskh,khadijah.abass@rutgers.edu,Khadijah,Abass,RESEARCH NURSE CLINICIAN,CINJ-OHRS Clinical RNC,1,STAFF,rbhs_faculty_staff
6,bla51,bla51@rutgers.edu,Bruno,Abbadi,POST DOC FELLOW,RWJ-Biochemistry&Molecular Bio,1,STAFF,rbhs_faculty_staff
7,khokhaar,khokhaar@rutgers.edu,Arshia,Abbasi,PHYSICIAN SPEC UNCLASSIFIED,NJMS-Clinical Affairs,1,STAFF,rbhs_faculty_staff
8,abbottgl,abbottgl@rutgers.edu,Gerald,Abbott,UCHC DENTIST I,UCHC-Bayside St Prsn-Ancillary,1,STAFF,rbhs_faculty_staff
9,ya296,ya296@rutgers.edu,Yazan,Abboud,HOUSESTAFF PGY II,NJMS H S - MEDICINE,1,STAFF,rbhs_faculty_staff


Cleaning PWAC file

In [3]:
# ============================
# Clean + Dedupe PWAC Residents file
# ============================

import pandas as pd
import numpy as np

# (1) Path to PWAC dataset
path_pwac = "/content/PWAC Resident Info 2024(PWAC Emails).csv"

# (2) Load with safe encoding and tidy headers
pwac_raw = pd.read_csv(path_pwac, encoding="latin1")
pwac_raw.columns = pwac_raw.columns.str.strip()
print(f"✅ Loaded PWAC file | Shape: {pwac_raw.shape}")

# (3) Normalize key fields (NetID + Email)
if "Netid" in pwac_raw.columns:
    pwac_raw["Netid"] = (
        pwac_raw["Netid"]
        .astype(str).str.strip().str.lower()
        .replace({"nan": pd.NA})
    )

# PWAC email column is "Organizational Email"
if "Organizational Email" in pwac_raw.columns:
    pwac_raw["Organizational Email"] = (
        pwac_raw["Organizational Email"]
        .astype(str).str.strip().str.lower()
        .replace({"nan": pd.NA})
    )

# (4) Normalize names
first = pwac_raw.get("First Name", pd.Series(dtype=object)).astype(str).str.strip().replace({"nan": pd.NA})
last  = pwac_raw.get("Last Name",  pd.Series(dtype=object)).astype(str).str.strip().replace({"nan": pd.NA})

# (5) Build cleaned DataFrame with clear prefixes for PWAC-specific fields
pwac_clean = pd.DataFrame({
    "netid":                  pwac_raw.get("Netid", pd.NA),
    "email":                  pwac_raw.get("Organizational Email", pd.NA),
    "firstname":              first,
    "lastname":               last,
    "pwac_institution":       pwac_raw.get("Institution", pd.NA),
    "pwac_current_program":   pwac_raw.get("Current Program", pd.NA),
    "pwac_status":            pwac_raw.get("Status", pd.NA),
    "pwac_work_roles":        pwac_raw.get("Work Roles", pd.NA),
    "pwac_program_start_date": pd.to_datetime(pwac_raw.get("Program Start Date", pd.NA), errors="coerce"),
    "pwac_program_end_date":   pd.to_datetime(pwac_raw.get("Program End Date", pd.NA), errors="coerce"),
    "pwac_post_graduate_year": pwac_raw.get("Post Graduate Year", pd.NA),
    "pwac_rms_id_number":      pwac_raw.get("RMS ID Number", pd.NA),
    "source":                 "pwac_residents"
})

# (6) Drop rows with no identifying info (no netid + no email + no name)
pwac_clean = pwac_clean[~(
    pwac_clean["netid"].isna() &
    pwac_clean["email"].isna() &
    pwac_clean["firstname"].isna() &
    pwac_clean["lastname"].isna()
)].copy()

# (7) Deduplicate:
#     - Prefer NetID as unique key
#     - Fallback to Email for rows without NetID
has_netid = pwac_clean["netid"].notna()
d_with_netid = pwac_clean[has_netid].drop_duplicates(subset=["netid"], keep="first")
d_no_netid   = pwac_clean[~has_netid]
if "email" in d_no_netid:
    d_no_netid = d_no_netid.drop_duplicates(subset=["email"], keep="first")

pwac_clean = pd.concat([d_with_netid, d_no_netid], ignore_index=True)
pwac_clean = pwac_clean.drop_duplicates().reset_index(drop=True)

# (8) Reorder columns to match ID-first pattern
ID_COLS = ["netid", "email", "firstname", "lastname"]
META    = [c for c in pwac_clean.columns if c not in ID_COLS]
pwac_clean = pwac_clean[ID_COLS + META]

# (9) Save + preview
save_path = "/content/pwac_cleaned.xlsx"
pwac_clean.to_excel(save_path, index=False)

print("✅ Cleaned PWAC shape:", pwac_clean.shape)
print("💾 Saved to:", save_path)
pwac_clean.head(10)


✅ Loaded PWAC file | Shape: (1785, 17)
✅ Cleaned PWAC shape: (1769, 13)
💾 Saved to: /content/pwac_cleaned.xlsx


,netid,email,firstname,lastname,pwac_institution,pwac_current_program,pwac_status,pwac_work_roles,pwac_program_start_date,pwac_program_end_date,pwac_post_graduate_year,pwac_rms_id_number,source
0,aa2513,ahmad.almelegy@rwjbh.org,Ahmad,Almelegy,Trinitas Regional Medical Center,TRI - Internal Medicine,PRG 1,Resident,2024-07-01,2025-06-30,1.0,51655,pwac_residents
1,jhb185,jae.byun@rwjbh.org,Jae Hyuk Byun,Byun,Trinitas Regional Medical Center,TRI - Internal Medicine,PRG 1,Resident,2024-07-01,2025-06-30,1.0,51658,pwac_residents
2,vg462,vaishnavi.gurumurthy@rwjbh.org,Vaishnavi,Gurumurthy,Trinitas Regional Medical Center,TRI - Internal Medicine,PRG 2,Resident,2023-07-01,2026-06-30,2.0,46344,pwac_residents
3,sc1738,syeda.chillious@rwjbh.org,Syeda,Chillious,Trinitas Regional Medical Center,TRI - Podiatry,PRG 1,Resident,2024-07-01,2027-06-30,1.0,51688,pwac_residents
4,cw1150,ceerra.williams@rwjbh.org,Ceearra,Williams,Trinitas Regional Medical Center,TRI - Podiatry,PRG 1,Resident,2024-07-01,2027-06-30,1.0,51687,pwac_residents
5,ma1890,murielle.ahawo@rwjbh.org,Murielle,Ahawo,Trinitas Regional Medical Center,TRI- Psychiatry,PRG 4,Resident,2021-07-01,2025-06-30,4.0,30596,pwac_residents
6,ae637,alexis.evangelista@rwjbh.org,Alexis,Evangelista,Trinitas Regional Medical Center,TRI- Psychiatry,PRG 1,Resident,2024-07-01,2028-06-30,1.0,51632,pwac_residents
7,pf302,pantea.faridnia@rwjbh.org,Pantea,Faridnia,Trinitas Regional Medical Center,TRI- Psychiatry,PRG 3,Resident,2024-07-01,2026-06-30,3.0,51819,pwac_residents
8,vg382,victoria.guzman@rwjbh.org,Victoria,Guzman,Trinitas Regional Medical Center,TRI- Psychiatry,PRG 4,Resident,2021-07-01,2025-06-30,4.0,30598,pwac_residents
9,gl656,gregory.lewis@rwjbh.org,Gregory,Lewis,Trinitas Regional Medical Center,TRI- Psychiatry,PRG 1,Resident,2024-07-01,2028-06-30,1.0,51634,pwac_residents


Merge

In [4]:
# ============================================
# Merge Postdocs + RBHS + PWAC → Rutgers_master
# ============================================

import pandas as pd
import numpy as np
import os

# --- Try to use in-memory DataFrames; if not present, load from files ---
def ensure_df(name, fallback_path):
    if name in globals():
        return globals()[name]
    if os.path.exists(fallback_path):
        if fallback_path.lower().endswith(".xlsx"):
            return pd.read_excel(fallback_path)
        else:
            return pd.read_csv(fallback_path, encoding="latin1")
    raise FileNotFoundError(f"Neither variable '{name}' nor file '{fallback_path}' found.")

postdocs_clean = ensure_df("postdocs_clean", "/content/postdocs_cleaned.xlsx")
rbhs_clean     = ensure_df("rbhs_clean",     "/content/rbhs_cleaned.xlsx")
pwac_clean     = ensure_df("pwac_clean",     "/content/pwac_cleaned.xlsx")

print("✅ Inputs")
print("postdocs_clean:", postdocs_clean.shape)
print("rbhs_clean    :", rbhs_clean.shape)
print("pwac_clean    :", pwac_clean.shape)

# --- Outer merge on netid ---
merged = postdocs_clean.merge(
    rbhs_clean, on="netid", how="outer", suffixes=("_postdoc", "_rbhs")
).merge(
    pwac_clean, on="netid", how="outer", suffixes=("", "_pwac")
)

# --- Coalesce core identity fields with priority: postdocs → rbhs → pwac ---
def coalesce3(a, b, c):
    return a.fillna(b).fillna(c)

merged["email"]     = coalesce3(merged.get("email_postdoc"),     merged.get("email_rbhs"),     merged.get("email"))
merged["firstname"] = coalesce3(merged.get("firstname_postdoc"), merged.get("firstname_rbhs"), merged.get("firstname"))
merged["lastname"]  = coalesce3(merged.get("lastname_postdoc"),  merged.get("lastname_rbhs"),  merged.get("lastname"))

# --- Build a simple source lineage for traceability (optional) ---
for s in ("source_postdoc", "source_rbhs", "source"):
    if s not in merged.columns:
        merged[s] = pd.NA

merged["source"] = (
    merged[["source_postdoc","source_rbhs","source"]]
      .astype(str)
      .replace({"<NA>": np.nan, "nan": np.nan})
      .agg(lambda r: ",".join([x for x in r if pd.notna(x)]), axis=1)
      .str.replace(r"(^,|,$)", "", regex=True)
)

# --- Drop helper columns created by merge for identity fields ---
cols_to_drop = [
    "email_postdoc","email_rbhs",
    "firstname_postdoc","firstname_rbhs",
    "lastname_postdoc","lastname_rbhs"
]
merged = merged.drop(columns=[c for c in cols_to_drop if c in merged.columns])

# NOTE: Attribute columns are already prefixed (postdoc_*, rbhs_*, pwac_*), so no conflicts.

# --- Final deduplication ---
# Prefer unique netid; for rows without netid, fall back to unique email.
has_netid = merged["netid"].notna()
master_with_netid = merged[has_netid].drop_duplicates(subset=["netid"], keep="first")
master_no_netid   = merged[~has_netid]
if "email" in master_no_netid.columns:
    master_no_netid = master_no_netid.drop_duplicates(subset=["email"], keep="first")

rutgers_master = pd.concat([master_with_netid, master_no_netid], ignore_index=True)
rutgers_master = rutgers_master.drop_duplicates().reset_index(drop=True)

# --- Order columns: IDs first, then everything else ---
ID_COLS = ["netid","email","firstname","lastname"]
OTHER   = [c for c in rutgers_master.columns if c not in ID_COLS]
rutgers_master = rutgers_master[ID_COLS + OTHER]

print("✅ Rutgers master shape:", rutgers_master.shape)

# --- Save full master ---
full_path = "/content/Rutgers_master.xlsx"
rutgers_master.to_excel(full_path, index=False)
print("💾 Saved full master to:", full_path)

# --- (Optional) also save a SLIM version with the most useful columns ---
SLIM_COLS = [c for c in [
    "netid","email","firstname","lastname",
    # Postdoc highlights
    "postdoc_title","postdoc_department","postdoc_reporting_campus",
    "postdoc_school_div","postdoc_hire_date","postdoc_fte",
    # RBHS highlights
    "rbhs_title","rbhs_department","rbhs_employee_type","rbhs_role",
    # PWAC highlights
    "pwac_current_program","pwac_status","pwac_work_roles",
    "pwac_program_start_date","pwac_program_end_date","pwac_post_graduate_year",
    # Provenance
    "source"
] if c in rutgers_master.columns]

rutgers_master_slim = rutgers_master[SLIM_COLS].copy()
slim_path = "/content/Rutgers_master_SLIM.xlsx"
rutgers_master_slim.to_excel(slim_path, index=False)
print("💾 Saved slim master to:", slim_path)

# --- Quick QA ---
print("\n--- QA ---")
print("Nulls in key fields:\n", rutgers_master[["netid","email","firstname","lastname"]].isna().sum())
dup_netid = rutgers_master["netid"].dropna().duplicated().sum()
no_netid  = rutgers_master["netid"].isna()
dup_email_no_netid = rutgers_master.loc[no_netid, "email"].dropna().duplicated().sum()
print("Duplicate netid rows:", dup_netid)
print("Duplicate email among rows with no netid:", dup_email_no_netid)


✅ Inputs
postdocs_clean: (591, 15)
rbhs_clean    : (10425, 9)
pwac_clean    : (1769, 13)
✅ Rutgers master shape: (12395, 29)
💾 Saved full master to: /content/Rutgers_master.xlsx
💾 Saved slim master to: /content/Rutgers_master_SLIM.xlsx

--- QA ---
Nulls in key fields:
 netid        1679
email           1
firstname       0
lastname        0
dtype: int64
Duplicate netid rows: 0
Duplicate email among rows with no netid: 0
